In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]

In [98]:
URL = 'https://en.wikipedia.org/wiki/Hazel_dormouse'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

textContent = {}


Species = soup.title\
                .string.split('-')[0]\
                .rstrip(' ')

for Tags in soup.find_all('h2'):
    #print(Tags)
    # Get the chapters
    Chapters = Tags.text.strip()
    #print(Chapters)
    # If chapter == description
    if Chapters == 'Description[edit]' or \
        Chapters == 'Characteristics[edit]' or \
        Chapters == 'Appearance[edit]':
        # Find the text
        for Text in Tags.find_next_siblings('p'):
            #print(Text)
            if Chapters in Text.find_previous_siblings('h2')[0].text.strip():
                textContent[Species] = Text.text.strip()

In [115]:
textContent

{'House mouse': ['House mice have an adult body length (nose to base of tail) of 7.5–10 centimetres (3–4\xa0in) and a tail length of 5–10\xa0cm (2–4\xa0in). The weight is typically 40–45\xa0g (1+3⁄8–1+5⁄8\xa0oz). In the wild they vary in color from grey and light brown to black (individual hairs are actually agouti coloured), but domesticated fancy mice and laboratory mice are produced in many colors ranging from white to champagne to black.[4] They have short hair and some, but not all, sub-species have a light belly.[4] The ears and tail have little hair. The hind feet are short compared to Apodemus mice, only 15–19\xa0mm (9⁄16–3⁄4\xa0in) long; the normal gait is a run with a stride of about 4.5\xa0cm (1+3⁄4\xa0in), though they can jump vertically up to 45\xa0cm (18\xa0in).[5] The voice is a high-pitched squeak.[6][7] House mice thrive under a variety of conditions; they are found in and around homes and commercial structures, as well as in open fields and agricultural lands.\n',
  '

In [118]:
URL = 'https://en.wikipedia.org/wiki/Field_vole'
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

Species = soup.title\
                .string.split('-')[0]\
                .rstrip(' ')

textContent = {}
textContent[Species] = []

for Tags in soup.find_all('h2')[1:-1]:
    # Set chapter variable
    Chapter = Tags.span.attrs['id']
    # Check if the chapter is description (or similar)
    if Chapter == 'Characteristics'or \
       Chapter == 'Description' or \
       Chapter == 'Appearance':
        
        # Get the next sibling (text)
        for Text in Tags.find_next_siblings('p'):
            # But only if chapter is still valid as previous sibling
            if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                textContent[Species].append(Text.text)


In [119]:
textContent

{'Field vole': ['The field vole is a small, dark brown rodent with a short tail, distinguishable from the closely related common vole (Microtus arvalis) by its darker, longer and shaggier hair and by its more densely haired ears. The head and body length varies between 8 and 13 centimetres (3.1 and 5.1\xa0in) and the tail between 3 and 4 centimetres (1.2 and 1.6\xa0in). The weight is 20 to 50 grams (0.71 to 1.76\xa0oz).[3] The voice is a faint, low squeak and it also emits a range of chattering sounds.[4]\n']}

In [45]:
textContent = {}

for WikiPage in AnimalsWikiPages:
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get species name
    Species = soup.title\
                .string.split('-')[0]\
                .rstrip(' ')
    
    for Tags in soup.find_all('h2')[1:]:
        # Get the chapters
        Chapters = Tags.text.strip()
        # If chapter == description
        if Chapters == 'Description[edit]':
            # Find the text
            for Text in Tags.find_next_siblings('p'):
                if Chapters in Text.find_previous_siblings('h2')[0].text.strip():
                    textContent[Species] = Text.text.strip()
    
    